<a href="https://colab.research.google.com/github/jawadbayg/LLM-py-ngrok/blob/main/LLM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Install the OpenAI package if it's not already installed
!pip install openai


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 362.9/362.9 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 17.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.4 MB/s eta 0:00:00


In [2]:
# Replace <YOUR Novita AI API Key> with your actual API key
client = OpenAI(
    base_url="https://api.novita.ai/v3/openai",
    api_key="4ac337f1-d7af-494a-9896-26cfccfcc314",
)


NameError: name 'OpenAI' is not defined

In [2]:
!pip install Flask


In [3]:
# Install Flask and Flask-Cors (to handle CORS issues)
!pip install Flask flask-cors

# Install Ngrok
!pip install pyngrok


In [4]:
from pyngrok import ngrok

# Replace 'YOUR_AUTHTOKEN' with your actual Ngrok authtoken
ngrok.set_auth_token('2M87w0uKCDqdkk8MpTOOPvlhptw_74Bk91eAUrNUgtpNEQqwn')


In [5]:
from pyngrok import ngrok

# Start Ngrok and expose the Flask app on port 5000
public_url = ngrok.connect(5000)
print(f"Your public API URL is: {public_url}")


Your public API URL is: NgrokTunnel: "https://a85d-34-125-236-208.ngrok-free.app" -> "http://localhost:5000"


In [6]:


import os
import threading
from flask import Flask, request, jsonify, render_template_string
from flask_cors import CORS
from openai import OpenAI
from pyngrok import ngrok

# Initialize the Flask app
app = Flask(__name__)
CORS(app)  # Enable CORS

# Initialize the OpenAI client
client = OpenAI(
    base_url="https://api.novita.ai/v3/openai",
    api_key="4ac337f1-d7af-494a-9896-26cfccfcc314",  # Replace with your API key
)

model = "meta-llama/llama-3.1-8b-instruct"
max_tokens = 512

@app.route('/')
def home():
    return render_template_string('''
        <html>
        <body>
            <h1>Welcome to the Flask and Ngrok Server!</h1>
            <p>Use the /chat endpoint to interact with the model.</p>
            <p>Example POST request:</p>
            <pre>
            curl -X POST http://<your-ngrok-url>/chat -H "Content-Type: application/json" -d '{"message": "Hello!"}'
            </pre>
        </body>
        </html>
    ''')

@app.route('/chat', methods=['POST'])
def chat():
    data = request.json
    user_input = data.get("message", "")
    conversation_history = [
        {
            "role": "system",
            "content": "Act like you are a blog content writer.",
        },
        {
            "role": "user",
            "content": user_input,
        }
    ]

    # Get the model response
    chat_completion_res = client.chat.completions.create(
        model=model,
        messages=conversation_history,
        stream=False,
        max_tokens=max_tokens,
    )

    assistant_response = chat_completion_res.choices[0].message.content

    return jsonify({"response": assistant_response})

def run_flask_app():
    app.run(host='0.0.0.0', port=5004)

# Start Flask server in a separate thread
flask_thread = threading.Thread(target=run_flask_app)
flask_thread.start()

# Set up Ngrok tunnel
ngrok.set_auth_token("2M87w0uKCDqdkk8MpTOOPvlhptw_74Bk91eAUrNUgtpNEQqwn")  # Replace with your Ngrok auth token
public_url = ngrok.connect(5004)
print(f"Ngrok public URL: {public_url}")

# The server and Ngrok tunnel are now set up


 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5004
 * Running on http://172.28.0.12:5004
INFO:werkzeug:Press CTRL+C to quit


Ngrok public URL: NgrokTunnel: "https://65d5-34-125-236-208.ngrok-free.app" -> "http://localhost:5004"
